Social media scraping
===

In [12]:
import os
import json

data_dir = "data/twitter"

if os.path.exists(data_dir) == False:
    os.makedirs(data_dir)

## Tweets by User

Scrape the last 100 tweets from `@derstandard` and store in json file

In [53]:
!snscrape --jsonl --max-results 100 twitter-user derstandardat > data/twitter/user-derstandardat.json


with open(data_dir+"/user-derstandardat.json", "r") as fo:
    derstandard_tweets = fo.readlines()

derstandard_tweets = [json.loads(derstandard_tweets[i]) for i in range(0, len(derstandard_tweets))]


print("loaded", len(derstandard_tweets), "tweets\n")


# print the first 5
for i in range(min(5, len(derstandard_tweets))):
    print(derstandard_tweets[i]["date"], derstandard_tweets[i]["rawContent"], derstandard_tweets[i]["hashtags"])

loaded 100 tweets

2023-01-22T19:49:33+00:00 XSudoku mittel 5422b:  https://t.co/JpypjJAZiU None
2023-01-22T19:17:17+00:00 Uhrmacher: "Die Kundschaft glaubt, mir die Arbeit erklären zu müssen":  https://t.co/XgBdv9G5Qs None
2023-01-22T18:02:02+00:00 Weiterarbeiten im Pensionsalter: "Meine Arbeit ist mein Leben":  https://t.co/C1zpXtuZ9j None
2023-01-22T18:02:01+00:00 Ältere sollen die Joblücke füllen. Aber wer will sie wirklich haben?:  https://t.co/a0yvyd7NAQ None
2023-01-22T17:31:23+00:00 Julian Schütter: Kitzbühel-Opfer und Skifahrer for Future:  https://t.co/YEY11CcFhs None


## Tweets by Hashtag

Scrape the last 100 tweets with `#chinesevirus` and store in json file

In [54]:
!snscrape --jsonl --max-results 100 twitter-hashtag chinesevirus > data/twitter/hashtag-chinesevirus.json


with open(data_dir+"/hashtag-chinesevirus.json", "r") as fo:
    tweets = fo.readlines()

tweets = [json.loads(tweets[i]) for i in range(0, len(tweets))]


print("loaded", len(tweets), "tweets\n")


# print the first 5
for i in range(min(5, len(tweets))):
    print(tweets[i]["date"], tweets[i]["rawContent"], tweets[i]["hashtags"])

loaded 100 tweets

2023-01-22T20:41:50+00:00 @EpochTimesEs #chinesevirus ['chinesevirus']
2023-01-22T18:41:36+00:00 #chineseVirus ['chineseVirus']
2023-01-22T15:30:19+00:00 #ChineseVirus update https://t.co/fkpN3sRSrL ['ChineseVirus']
2023-01-22T15:02:11+00:00 @AmbassadeChine Year of Chinese BAT
#chinesevirus ['chinesevirus']
2023-01-22T13:10:58+00:00 #ChineseNewYear #ChineseVirus ['ChineseNewYear', 'ChineseVirus']


In [52]:
!snscrape --jsonl --max-results 100 twitter-hashtag schlitzaugenvirus > data/twitter/hashtag-schlitzaugenvirus.json


with open(data_dir+"/hashtag-schlitzaugenvirus.json", "r") as fo:
    tweets = fo.readlines()

tweets = [json.loads(tweets[i]) for i in range(0, len(tweets))]


print("loaded", len(tweets), "tweets\n")


# print the first 5
for i in range(min(5, len(tweets))):
    print(tweets[i]["date"], tweets[i]["rawContent"], tweets[i]["hashtags"])

loaded 3 tweets

2020-10-20T21:34:56+00:00 #Plandemie ist kein #SchlitzaugenVirus @StefanPetzner im Interview https://t.co/7zb8NN9NF4 ['Plandemie', 'SchlitzaugenVirus']
2020-08-17T07:39:46+00:00 @stefan_petzner @cbischofberger Mit rassistischen Sagern würde man „die Menschen“ wohl besser erreichen, was? #schlitzaugenvirus ['schlitzaugenvirus']
2020-06-03T16:35:49+00:00 Man muss schon ein extremer Realitätsverweigerer sein, wenn man sein Profil, trotz des längst gescheiterten schwedischen Weges, mit einer Schwedenflagge schmückt.

Oder hat es gar mit schwedischen Gardinen zu tun?
#SchlitzaugenVirus https://t.co/uZpbYtOnpN ['SchlitzaugenVirus']


In [51]:
!snscrape --jsonl --max-results 100 twitter-hashtag plandemie > data/twitter/hashtag-plandemie.json


with open(data_dir+"/hashtag-plandemie.json", "r") as fo:
    tweets = fo.readlines()

tweets = [json.loads(tweets[i]) for i in range(0, len(tweets))]


print("loaded", len(tweets), "tweets\n")


# print the first 5
for i in range(min(5, len(tweets))):
    print(tweets[i]["date"], tweets[i]["rawContent"], tweets[i]["hashtags"])

loaded 100 tweets

2023-01-22T20:52:27+00:00 Gestern in der Sauna. 
Bin ja  als #Ungeimpfter jeden weiteren Tag entspannter und gelassener. 
Wenn aber ein Honk behauptet, dass die #Plandemie nur deswegen so massiv war, weil die #Ungeimpften die #Geimpften infiziert haben, kann ich nicht anders als zu reagieren. ['Ungeimpfter', 'Plandemie', 'Ungeimpften', 'Geimpften']
2023-01-22T20:39:46+00:00 #LucMontagnier #VIH #Covid
#Plandemic #plandemia #plandemie #plandemi https://t.co/HwILgcTnNS ['LucMontagnier', 'VIH', 'Covid', 'Plandemic', 'plandemia', 'plandemie', 'plandemi']
2023-01-22T20:39:06+00:00 Diesem Mann sollte man zuhören. Er hat die Farce der #Schweinegrippe bereits durchschaut. Mit ihm als #Gesundheitsminister hätte es die #PLANdemie niemals gegeben. Lest auch sein Buch: Falsche Pandemien. #Wodarg #LongCovid #Impfschäden ['Schweinegrippe', 'Gesundheitsminister', 'PLANdemie', 'Wodarg', 'LongCovid', 'Impfschäden']
2023-01-22T18:12:12+00:00 Ein #Vorteil der #Corona-#Plandemie ist es, 

All

In [ ]:
os.call()

In [ ]:
!snscrape --jsonl twitter-hashtag plandemic > data/twitter/hashtag-plandemic.json


with open(data_dir+"/hashtag-plandemic.json", "r") as fo:
    tweets = fo.readlines()

tweets = [json.loads(tweets[i]) for i in range(0, len(tweets))]


print("loaded", len(tweets), "tweets\n")


# print the first 5
for i in range(min(5, len(tweets))):
    print(tweets[i]["date"], tweets[i]["rawContent"], tweets[i]["hashtags"])